In [6]:
import os 
import random
import shutil
from pathlib import Path

In [3]:
def count_images_in_folders(base_dir):
    for split in os.listdir(base_dir):
            split_path = os.path.join(base_dir, split)
            if os.path.isdir(split_path):
                  print(f"\n Split: {split}")
                  for emotion in os.listdir(split_path):
                        emotion_path = os.path.join(split_path,emotion)
                        if os.path.isdir(emotion_path):
                              count = len([
                                    f for f in os.listdir(emotion_path)
                                    if os.path.isfile(os.path.join(emotion_path,f))
                              ])
                              print(f" {emotion}: {count} images")

In [10]:
count_images_in_folders("fer2013_data")  


 Split: val
 fear: 409 images
 surprise: 317 images
 sad: 483 images
 happy: 721 images
 disgust: 43 images
 neutral: 496 images
 angry: 399 images

 Split: train
 fear: 3688 images
 surprise: 2854 images
 sad: 4347 images
 happy: 6494 images
 disgust: 393 images
 neutral: 4469 images
 angry: 3596 images

 Split: test
 fear: 1024 images
 surprise: 831 images
 sad: 1247 images
 happy: 1774 images
 disgust: 111 images
 neutral: 1233 images
 angry: 958 images


In [7]:
def split_train_val(train_dir, val_dir, val_ratio = 0.1, seed = 42):
    random.seed(seed)

    Path(val_dir).mkdir(parents=True, exist_ok=True)

    for class_name in os.listdir(train_dir):
        class_path = os.path.join(train_dir,class_name)
        if not os.path.isdir(class_path):
            continue
        
        images = os.listdir(class_path)
        random.shuffle(images)

        n_val = int(len(images) * val_ratio)
        val_images = images[:n_val]

        val_class_path = os.path.join(val_dir, class_name)
        Path(val_class_path).mkdir(parents=True, exist_ok= True)

        for img_name in val_images:
            src_path = os.path.join(class_path,img_name)
            dst_path = os.path.join(val_class_path, img_name)
            shutil.move(src_path,dst_path)

        print(f"{class_name}: moved {n_val} images to validation set")

In [ ]:
# split_train_val(
#     train_dir='fer2013_data/train',
#     val_dir='fer2013_data/val',
#     val_ratio=0.1,
# )

fear: moved 409 images to validation set
surprise: moved 317 images to validation set
sad: moved 483 images to validation set
happy: moved 721 images to validation set
disgust: moved 43 images to validation set
neutral: moved 496 images to validation set
angry: moved 399 images to validation set
